In [20]:
import sys
import torch
sys.path.append('../src')
%load_ext autoreload
%autoreload 2

from generator import *
from evaluation import *
from fair_model import FairModel
from baselines import LR, CvxFairModel, EOFairModel
from utils import gen_plot_data, plot_axes, combine_tuples

In [6]:
# Build Bank model
bank = Bank()
agent_train = Agent(n_samples=5000, protect_ratio=0.5, eps=0.5, base=[0.2, 1.0], seed=2021)
agent_test = Agent(n_samples=1000, protect_ratio=0.5, eps=0.5, base=[0.2, 1.0], seed=2020)

In [7]:
s_train, Xs_train, Ds_train, Ys_train = gen_multi_step_process(bank, agent_train, steps=5)
s_test, Xs_test, Ds_test, Ys_test = gen_multi_step_process(bank, agent_test, steps=5)
s_comb, X_comb, Y_comb = combine_tuples(s_train, Xs_train, Ys_train)

In [8]:
# Generate temporal graph with opt-out and save a full panel CSV (includes neighbor decision vector D1..D10)
steps = 5

s, adj, edges, Xs, Ds, Ys, Ps, Os, Us, As = gen_temporal_graph_with_optout(
    bank,
    agent_train,
    steps,
    graph_seed=2021,
    seed=2021,
    spillover_threshold=0.6,
    enforce_demographic_mixing=True,
    k_same=6,
    k_other=4,
    decision_coef=0.8,
    repayment_coef=0.8,
    decision_positive_is_approval=True,
 )

out_path = "../src/simulation_results_neighbors.csv"
save_agent_panel_csv(
    out_path,
    s=s,
    Xs=Xs,
    adj=adj,
    Ds=Ds,
    Ys=Ys,
    Ps=Ps,
    Us=Us,
    As=As,
    Os=Os,
    t0=0,
    neighbor_k=10,
 )

print("Wrote:", out_path)

Wrote: ../src/simulation_results_neighbors.csv


In [9]:
import numpy as np

def fpr_by_group(D_t, Y_t, s, *, group_value, A_t=None):
    """FPR here matches your formula: P(D_t=1 | S=group, Y_t=0).
    If A_t is provided, computes it among active agents only (A_t==1).
    """
    D_t = np.asarray(D_t).astype(int)
    Y_t = np.asarray(Y_t).astype(int)
    s = np.asarray(s).astype(int)
    mask = (s == int(group_value)) & (Y_t == 0)
    if A_t is not None:
        A_t = np.asarray(A_t).astype(int)
        mask = mask & (A_t == 1)
    denom = mask.sum()
    if denom == 0:
        return np.nan
    return float(D_t[mask].mean())

# Interpret s^+ as S=1 and s^- as S=0 (swap if your convention differs)
pe_active = []
pe_all = []

for t in range(steps):
    fpr1_all = fpr_by_group(Ds[t], Ys[t], s, group_value=1, A_t=None)
    fpr0_all = fpr_by_group(Ds[t], Ys[t], s, group_value=0, A_t=None)
    pe_t_all = abs(fpr1_all - fpr0_all)
    pe_all.append(pe_t_all)

    fpr1_act = fpr_by_group(Ds[t], Ys[t], s, group_value=1, A_t=As[t])
    fpr0_act = fpr_by_group(Ds[t], Ys[t], s, group_value=0, A_t=As[t])
    pe_t_act = abs(fpr1_act - fpr0_act)
    pe_active.append(pe_t_act)

    print(f"t={t}: FPR_all(S=1)={fpr1_all:.3f}, FPR_all(S=0)={fpr0_all:.3f}, PE_all={pe_t_all:.3f}")
    print(f"     FPR_active(S=1)={fpr1_act:.3f}, FPR_active(S=0)={fpr0_act:.3f}, PE_active={pe_t_act:.3f}")

print("PE_all:", pe_all)
print("PE_active:", pe_active)

t=0: FPR_all(S=1)=0.461, FPR_all(S=0)=0.528, PE_all=0.067
     FPR_active(S=1)=0.461, FPR_active(S=0)=0.528, PE_active=0.067
t=1: FPR_all(S=1)=0.187, FPR_all(S=0)=0.060, PE_all=0.127
     FPR_active(S=1)=0.369, FPR_active(S=0)=0.442, PE_active=0.073
t=2: FPR_all(S=1)=0.082, FPR_all(S=0)=0.035, PE_all=0.047
     FPR_active(S=1)=0.275, FPR_active(S=0)=0.413, PE_active=0.138
t=3: FPR_all(S=1)=0.063, FPR_all(S=0)=0.018, PE_all=0.045
     FPR_active(S=1)=0.281, FPR_active(S=0)=0.312, PE_active=0.031
t=4: FPR_all(S=1)=0.055, FPR_all(S=0)=0.012, PE_all=0.042
     FPR_active(S=1)=0.288, FPR_active(S=0)=0.250, PE_active=0.038
PE_all: [0.06735772357723574, 0.12704038119762523, 0.04695720869834994, 0.04478724878724878, 0.042489352377313536]
PE_active: [0.06735772357723574, 0.07290934016808726, 0.13793532338308456, 0.030717708874646366, 0.038288288288288286]


In [13]:
lr = LR(l2_reg=1e-5)
lr.train(s_comb, X_comb, Y_comb)

_, Xs_te, _, Ys_te = gen_multi_step_process(lr, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, lr, OYs=OYs_te)

------------------------------ Step 1 - Logistic Regression ------------------------------
Acc: 87.1%
Short Fairness: 0.108
Long fairness: 0.050
------------------------------ Step 2 - Logistic Regression ------------------------------
Acc: 86.6%
Short Fairness: 0.124
Long fairness: 0.113
------------------------------ Step 3 - Logistic Regression ------------------------------
Acc: 87.8%
Short Fairness: 0.124
Long fairness: 0.178
------------------------------ Step 4 - Logistic Regression ------------------------------
Acc: 89.3%
Short Fairness: 0.130
Long fairness: 0.193
------------------------------ Step 5 - Logistic Regression ------------------------------
Acc: 89.6%
Short Fairness: 0.152
Long fairness: 0.303




In [25]:
d = X_comb.shape[1]            # number of X columns
cfm = CvxFairModel(n_features=d + 1, l2_reg=1e-5, tao=1.565)  # s + X

# cfm = CvxFairModel(n_features=len(Xs_train[0][0])+2, l2_reg=1e-5, tao=1.565)

cfm.train(s_comb, X_comb, Y_comb)



In [27]:
_, Xs_te,_, Ys_te = gen_multi_step_process(cfm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, cfm, OYs=OYs_te) 

------------------------------ Step 1 - Fair Model with Demographic Parity ------------------------------
Acc: 64.9%
Short Fairness: 0.000
Long fairness: 0.000
------------------------------ Step 2 - Fair Model with Demographic Parity ------------------------------
Acc: 58.6%
Short Fairness: 0.000
Long fairness: 0.001
------------------------------ Step 3 - Fair Model with Demographic Parity ------------------------------
Acc: 55.1%
Short Fairness: 0.000
Long fairness: 0.002
------------------------------ Step 4 - Fair Model with Demographic Parity ------------------------------
Acc: 50.6%
Short Fairness: 0.000
Long fairness: 0.003
------------------------------ Step 5 - Fair Model with Demographic Parity ------------------------------
Acc: 46.9%
Short Fairness: 0.000
Long fairness: 0.004




In [18]:
eqm = EOFairModel(n_features=len(Xs_train[0][0])+2, l2_reg=1e-5, tao=1.5)
eqm.train(s_comb, X_comb, Y_comb)

_, Xs_te, _, Ys_te = gen_multi_step_process(eqm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, eqm, OYs=OYs_te)

optimal
------------------------------ Step 1 - Fair Model with Equal Oppertunity ------------------------------
Acc: 79.5%
Short Fairness: 0.046
Long fairness: 0.104
------------------------------ Step 2 - Fair Model with Equal Oppertunity ------------------------------
Acc: 76.7%
Short Fairness: 0.048
Long fairness: 0.176
------------------------------ Step 3 - Fair Model with Equal Oppertunity ------------------------------
Acc: 76.1%
Short Fairness: 0.056
Long fairness: 0.265
------------------------------ Step 4 - Fair Model with Equal Oppertunity ------------------------------
Acc: 75.9%
Short Fairness: 0.066
Long fairness: 0.349
------------------------------ Step 5 - Fair Model with Equal Oppertunity ------------------------------
Acc: 77.5%
Short Fairness: 0.068
Long fairness: 0.465




In [22]:
fm = FairModel(n_features=len(Xs_train[0][0])+1, lr=5e-3, l2_reg=1e-5, sf_reg=0.119, lf_reg=0.154)
fm.train(s_train, Xs_train, Ys_train, Xs_train, Ys_train, epochs=1000, plot=False)

num_iters = 50

theta_true = fm.params
theta_list     = [np.copy(theta_true)]
theta_gaps     = []


# inital theta
theta = np.copy(theta_true)

for t in range(num_iters):
    # adjust distribution to current theta
    _, NXs_train, _, NYs_train = gen_multi_step_process(fm, agent_train, steps=5)
    # learn on induced distribution
    fm.train(s_train, Xs_train, Ys_train, NXs_train, NYs_train, epochs=10, plot=False)
    
    # keep track of statistic
    theta_new = fm.params
    theta_gaps.append(np.linalg.norm(theta_new - theta))
    theta_list.append(np.copy(theta_new))

    theta = np.copy(theta_new)
print("Retraining Done!")

Retraining Done!


In [24]:
_, Xs_te,Ys_te = gen_multi_step_profiles(fm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, fm, OYs=OYs_te)

------------------------------ Step 1 - Long-term Fair Model ------------------------------
Acc: 87.4%
Short Fairness: 0.228
Long fairness: 0.082
------------------------------ Step 2 - Long-term Fair Model ------------------------------
Acc: 87.7%
Short Fairness: 0.230
Long fairness: 0.146
------------------------------ Step 3 - Long-term Fair Model ------------------------------
Acc: 85.1%
Short Fairness: 0.246
Long fairness: 0.276
------------------------------ Step 4 - Long-term Fair Model ------------------------------
Acc: 84.4%
Short Fairness: 0.258
Long fairness: 0.370
------------------------------ Step 5 - Long-term Fair Model ------------------------------
Acc: 85.6%
Short Fairness: 0.266
Long fairness: 0.525


